# LangChain Agents and Llama Stack with MCP Tools

This notebook demonstrates **LangChain 1.0** integration with **Llama Stack** responses API using proper **MCP (Model Context Protocol) adapters** for client-side tool execution.

## Configuration

This notebook uses environment variables from `.env` file in the project root.
Create your own `.env` file based on `.env.example`.

## Architecture

```
┌─────────────────────────────────────────────┐
│         LangChain 1.0 Agent                 │
│         (create_agent)                      │
└────────┬────────────────────┬───────────────┘
         │                    │
         │ Model Calls        │ Tool Execution
         ▼                    ▼
┌─────────────────┐   ┌──────────────────────┐
│  Llama Stack    │   │  MCP Client          │
│  (OpenAI API)   │   │  (langchain-mcp)     │
│  - vLLM         │   │  - SSE Transport     │
│  - Llama 3.2    │   │  - Tool Adapters     │
└─────────────────┘   └──────┬───────────────┘
                             │
                             ▼
                      ┌──────────────┐
                      │ MCP Server   │
                      │ (Weather)    │
                      └──────────────┘
```

## Install Required Packages

Install LangChain 1.0 and MCP adapters:

In [ ]:
!pip3 install -q "langchain>=1.0" "langchain-openai>=0.3.32" "langchain-core>=0.3.75" "langchain-mcp-adapters>=0.1.0"

## Import Dependencies

In [ ]:
# Core imports
import os
import sys
import asyncio
from pathlib import Path
from pprint import pprint
from dotenv import load_dotenv

# LangChain 1.0 imports
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# MCP Adapters for client-side tool execution
from langchain_mcp_adapters.client import MultiServerMCPClient

# Load environment variables from project root
project_root = Path.cwd().parent.parent if 'assets/notebooks' in str(Path.cwd()) else Path.cwd()
env_path = project_root / '.env'
load_dotenv(env_path)

print(f"📁 Loading environment from: {env_path}")
print(f"✅ .env file {'found' if env_path.exists() else 'NOT FOUND'}")
print("✅ All dependencies imported successfully")

## Configure Llama Stack Connection

Connect to Llama Stack's OpenAI-compatible endpoint:

In [ ]:
# === Llama Stack Configuration ===
# Load from environment variables
LLAMA_STACK_OPENAI_ENDPOINT = os.getenv("LLAMA_STACK_OPENAI_ENDPOINT")
INFERENCE_MODEL = os.getenv("INFERENCE_MODEL")
API_KEY = os.getenv("API_KEY", "fake")

print("🌐 Llama Stack Configuration:")
print(f"   Endpoint using Responses API: {LLAMA_STACK_OPENAI_ENDPOINT}")
print(f"   Model: {INFERENCE_MODEL}")

# Create ChatOpenAI client pointing to Llama Stack
llm = ChatOpenAI(
    model=INFERENCE_MODEL,
    api_key=API_KEY,
    base_url=LLAMA_STACK_OPENAI_ENDPOINT,
    temperature=0.0,
)

print("\n🧪 Testing connectivity...")
try:
    response = llm.invoke("Say 'Connection successful' if you can read this.")
    print(f"📥 LLM Response: {response.content}")
    print(f"📥 LLM Response metadata: {response.response_metadata}")
    print("✅ Llama Stack connection successful!")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    sys.exit(1)

## Set Up MCP Client and Tools

Configure the MCP client to connect to the weather service using SSE transport:

In [ ]:
# === MCP Client Configuration ===
from langchain_mcp_adapters.client import MultiServerMCPClient

# Load MCP server URL from environment
MCP_WEATHER_SERVER_URL = os.getenv("MCP_WEATHER_SERVER_URL")

print("🛠️  Configuring MCP client...\n")
print(f"🌦️  MCP Server URL: {MCP_WEATHER_SERVER_URL}")

# Step 1: Create the MCP client
client = MultiServerMCPClient({
    "weather": {
        "transport": "sse",
        "url": MCP_WEATHER_SERVER_URL,
    }
})

# Step 2: Get tools from the MCP server (async -> await)
tools = await client.get_tools()

# Step 3: Show what we got
print(f"\n📦 Loaded {len(tools)} tools from MCP server:")
for tool in tools:
    print(f"   - {tool.name}: {tool.description}")

print("\n✅ MCP tools configured successfully!")

# We keep 'client' alive so tools can work later
mcp_client = client

## Create LangChain Agent

Use the new `create_agent` API from LangChain 1.0:

In [ ]:
# === Create Agent using LangChain v1.0 ===
print("🤖 Creating LangChain agent...\n")

# Create agent with proper configuration
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="""
You are a helpful weather assistant powered by Llama Stack.

You have access to weather tools that can retrieve current weather information.
When a user asks about weather, use the available tools to get accurate data.

Always:
- Be concise and friendly
- Use tools when needed to get real data
- Provide clear, actionable information
""".strip(),
)

print("✅ Agent created successfully!")
print("\n📊 Agent Configuration:")
print(f"   Model: {INFERENCE_MODEL}")
print(f"   Tools: {len(tools)} MCP tools available")
print(f"   Framework: LangChain 1.0 (create_agent)")

## Test the Agent

Let's test the agent with a weather query:

In [ ]:
# === Test Agent ===
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

print("\n" + "="*60)
print("🚀 Testing LangChain 1.0 Agent with MCP Tools")
print("="*60 + "\n")

# Define async function to run agent
async def run_agent_query(query: str):
    """
    Run agent query and display results.
    """
    print(f"👤 User: {query}\n")
    
    # Invoke agent
    result = await agent.ainvoke({
        "messages": [{"role": "user", "content": query}]
    })
    
    # Display results
    print("🤖 Agent Execution Trace:")
    print("-" * 60)
    
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"\n👤 Human: {message.content}")
        
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"\n🤖 AI (Tool Call):")
                for tool_call in message.tool_calls:
                    print(f"   Tool Calling: {tool_call['name']}")
                    print(f"   Args: {tool_call['args']}")
            else:
                print(f"\n🤖 AI: {message.content}")
        
        elif isinstance(message, ToolMessage):
            print(f"\n🛠️  Tool Result: {message.content}")
    
    print("\n" + "="*60)
    print("✅ Query completed successfully!")
    print("="*60)
    
    return result

# Run test query
result = await run_agent_query("What is the weather in Miami?")